<a href="https://colab.research.google.com/github/reganandrews/DeepCCA/blob/master/Regan_Andrews_Deep_Correlation_Learning_for_Urban_Air_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Correlation Learning for Urban Air Quality

This notebook demonstrates the workings of an architecture for deep correlation learning for urban air quality. Submitted in partial fulfilment for the degree of Doctor of Computing by Regan Andrews.

Firstly, the environment is setup.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import seaborn as sns
from string import ascii_letters
import IPython

import pickle

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## Importing the Dataset
This architecture uses a dataset created from data provided by the Auckland Council. It contains 9 different features such as atmospheric chemistry, air temperature, solar radiation and humidity. These were collected hourly, beginning in 2012.

All data from this dataset can be re-collected from the Auckland Council's Environment Auckland website: https://environmentauckland.org.nz/

Locations are indicated in this map: (link here)


In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://github.com/reganandrews/airdata/raw/master/site-gleneden.csv.zip',
    fname='site-gleneden.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [0]:
df = pd.read_csv(csv_path)

On viewing the imported data we see it has the following types of data:

In [0]:
df.head()

### Exploration of the data

We are interested in the correlative aspects of this data. Let's have a look and see what commonly accepted correlative measurements uncover.

In [0]:
corr_matrix = df.corr()
print(corr_matrix)


In [0]:
corr = df.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [0]:
sns.set(style="white")

corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [0]:
f = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=14, rotation=45)
plt.yticks(range(df.shape[1]), df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [0]:
#rs = np.random.RandomState(0)
#df = pd.DataFrame(rs.rand(10, 10))
#corr = df.corr()
#corr.style.background_gradient(cmap='coolwarm')

In [0]:
df.hist()
plt.show()

As you can see above, an observation is recorded every hour. This means that, for a single hour, you will have 9 observations. Similarly, a single day will contain 24 (9x24) observations. 

Given a specific time, let's assume we want to predict the temperature 6 hours in the future. In order to make this prediction, you choose to use 5 days of observations. Thus, you would create a window containing the last 720(5x144) observations to train the model. Many such configurations are possible, making this dataset a good one to experiment with.

The function below returns the above described windows of time for the model to train on. The parameter `history_size` is the size of the past window of information. The `target_size` is how far in the future does the model need to learn to predict. The `target_size` is the label that needs to be predicted.

In [0]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

In all of the following the first 100,000 rows of the data will be the training dataset, and the remaining will be the validation dataset.

In [0]:
TRAIN_SPLIT = 100000

We will set the seed to ensure reproducibility.

In [0]:
tf.random.set_seed(13)

## 1) Forecasting a univariate time-series (Temperature) for a single location (Glen Eden)
First, you will train a model using only a single feature (temperature), and use it to make predictions for that value in the future.

Let's first extract only the temperature from the dataset.

In [0]:
uni_data = df['gleneden.temp']
uni_data.index = df['DateTime']
uni_data.head()

Let's observe how this data looks across time.

In [0]:
uni_data.plot(subplots=True)

In [0]:
uni_data = uni_data.values

It is important to normalize features before training a neural network. A common way to do so is by subtracting the mean and dividing by the standard deviation of each feature.

Note: The mean and standard deviation should only be computed using the training data.

In [0]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

Let's normalize the data.

In [0]:
uni_data = (uni_data-uni_train_mean)/uni_train_std

Let's now create the data for the univariate model. For part 1, the model will be given the last 20 recorded temperature observations, and needs to learn to predict the temperature at the next time step. 

In [0]:
univariate_past_history = 20
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           20,
                                           0)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       20,
                                       0)

This is what the `univariate_data` function returns.

In [0]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target temperature to predict')
print (y_train_uni[0])

Now that the data has been created, let's take a look at a single example. The information given to the network is given in blue, and it must predict the value at the red cross.

In [0]:
def create_time_steps(length):
  time_steps = []
  for i in range(-length, 0, 1):
    time_steps.append(i)
  return time_steps

In [0]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [0]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example')

### Baseline
Before proceeding to train a model, let's first set a simple baseline. Given an input point, the baseline method looks at all the history and predicts the next point to be the average of the last 20 observations.

In [0]:
def baseline(history):
  return np.mean(history)

In [0]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example')

Let's see if you can beat this baseline using a recurrent neural network.

### Recurrent neural network

A Recurrent Neural Network (RNN) is a type of neural network well-suited to time series data. RNNs process a time series step-by-step, maintaining an internal state summarizing the information they've seen so far. For more details, read the [RNN tutorial](https://www.tensorflow.org/tutorials/sequences/recurrent). In this tutorial, you will use a specialized RNN layer called Long Short Term Memory ([LSTM](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/LSTM))

Let's now use `tf.data` to shuffle, batch, and cache the dataset.

In [0]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

The following visualisation should help you understand how the data is represented after batching.

![Time Series](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/time_series.png?raw=1)

You will see the LSTM requires the input shape of the data it is being given.

In [0]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')

Let's make a sample prediction, to check the output of the model. 

In [0]:
for x, y in val_univariate.take(1):
    print(simple_lstm_model.predict(x).shape)

Let's train the model now:

In [0]:
EVALUATION_INTERVAL = 50
EPOCHS = 20

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

simple_lstm_model.save(f"/content/gdrive/My Drive/Colab_Files//Models/simple_lstm_model.model")

#### Predict using the simple LSTM model
Now the simple LSTM model has been trained, we'll make some predictions:

In [0]:
for x, y in val_univariate.take(3):
  plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
  plot.show()

This looks better than the baseline. We will now work with a multivariate time series apporach for this single location.

## 2) Forecasting a multivariate time-series for a single location (Glen Eden)

The original dataset contains 9 features which we'll use now to forecast at a single site - Glen Eden. The features to be used are:


1. NO
2. NO2
3. NOx
4. PM10
5. Humidity
6. Solar Radiation
7. Temperature
8. Wind
9. Wind Speed


In [0]:
features_considered = ['gleneden.no', 'gleneden.no2', 'gleneden.nox', 'geneden.pm10',	'gleneden.hum',	'gleneden.solar',	'gleneden.temp',	'gleneden.wind',	'gleneden.ws']

In [0]:
features = df[features_considered]
features.index = df['DateTime']
features.head()

Let's have a look at how each of these features vary across time.

In [0]:
features.plot(subplots=True)

As mentioned, the first step will be to normalize the dataset using the mean and standard deviation of the training data.

In [0]:
dataset = features.values
data_mean = dataset.mean(axis=0)
data_std = dataset.std(axis=0)

In [0]:
dataset = (dataset-data_mean)/data_std

### Single step model
In a single step setup, the model learns to predict a single point in the future based on some history provided.

The below function performs the same windowing task as below, however, here it samples the past observation based on the step size given.

In [0]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

The network is shown data from the last 720 observations that are sampled hourly. The sampling is done every one hour since a drastic change is not expected within 60 minutes. 

In [0]:
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

Let's look at a single data-point.


In [0]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [0]:
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [0]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

Let's check out a sample prediction.

In [0]:
for x, y in val_data_single.take(1):
  print(single_step_model.predict(x).shape)

In [0]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

In [0]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [0]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

#### Predict a single step future
Now that the model is trained, let's make a few sample predictions. The model is given the history of three features over the past five days sampled every hour (120 data-points), since the goal is to predict the temperature, the plot only displays the past temperature. The prediction is made one day into the future (hence the gap between the history and prediction). 

In [0]:
for x, y in val_data_single.take(3):
  plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Single Step Prediction')
  plot.show()

### Multi-Step model
In a multi-step prediction model, given a past history, the model needs to learn to predict a range of future values. Thus, unlike a single step model, where only a single future point is predicted, a multi-step model predict a sequence of the future.

For the multi-step model, the training data again consists of recordings over the past five days sampled every hour. However, here, the model needs to learn to predict the temperature for the next 12 hours. Since an obversation is taken every 10 minutes, the output is 72 predictions. For this task, the dataset needs to be prepared accordingly, thus the first step is just to create it again, but with a different target window.

In [0]:
future_target = 72
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

Let's check out a sample data-point.

In [0]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train_multi[0].shape))

In [0]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

Plotting a sample data-point.

In [0]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

In this plot and subsequent similar plots, the history and the future data are sampled every hour.

In [0]:
for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

Since the task here is a bit more complicated than the previous task, the model now consists of two LSTM layers. Finally, since 72 predictions are made, the dense layer outputs 72 predictions.

In [0]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(72))

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')

Let's see how the model predicts before it trains.

In [0]:
for x, y in val_data_multi.take(1):
  print (multi_step_model.predict(x).shape)

In [0]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [0]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

#### Predict a multi-step future
Let's now have a look at how well your network has learnt to predict the future.

In [0]:
for x, y in val_data_multi.take(3):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

## 3) Which locations have a correlation?

The original dataset contains x locations, with a total of 109 features.

If we can use this dataset to determine which locations are closely correlated to 1 target location, we should be able to more accurately predict the results at that target location by the relevent data.

We'll start by looking at whether the chemical 'NO' and seeing what locations show correlation:

In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://github.com/reganandrews/airdata/raw/master/NO.csv.zip',
    fname='NO.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [0]:
df = pd.read_csv(csv_path)

In [0]:
df.head()

The correlation matrix for the master data shows the correlations that exist between all locations.

In [0]:
corr_matrix = df.corr()
print(corr_matrix)

What about another parameter, are the correlation values still similar? Let's try NO2:

In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://github.com/reganandrews/airdata/raw/master/NO2.csv.zip',
    fname='NO2.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [0]:
df = pd.read_csv(csv_path,)

In [0]:
df.head()

In [0]:
corr_matrix = df.corr()
print(corr_matrix)

## 4) Forecasting a multi-variate time-series for one location using the surrounding locations data

The original dataset contains x locations, with a total of 109 features which we'll use now to forecast everything at once.

We'll import this new data now from GitHub:


In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://github.com/reganandrews/airdata/raw/master/master_data_fixed.csv.zip',
    fname='master_data_fixed.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [0]:
df = pd.read_csv(csv_path)

In [0]:
df.head()

In [0]:
features_considered = ['gleneden.co', 'henderson.co', 'khyberpass.co', 'pakuranga.co', 'poal.co', 'queenst.co', 'takapuna.co', 'gleneden.no', 'gleneden.no2','gleneden.nox', 'henderson.no', 'henderson.no2', 'henderson.nox', 'musickpt.no', 'musickpt.no2', 'musickpt.nox', 'patumahoe.no', 'patumahoe.no2', 'patumahoe.nox', 'penrose.no', 'penrose.no2', 'penrose.nox', 'poal.no', 'poal.no2', 'poal.nox', 'queenst.no', 'queenst.no2', 'queenst.nox', 'takapuna.no', 'takapuna.no2', 'takapuna.nox', 'musickpt.o3', 'patumahoe.o3', 'whangarei.o3', 'botanydowns.pm10', 'geneden.pm10', 'henderson.pm10', 'khyberpass.pm10', 'kumeu.pm10', 'orewa.pm10', 'pakuranga.pm10', 'patumahoe.pm10', 'patumahoe.pm25', 'penrose.pm1.grimm', 'penrose.pm10', 'penrose.pm10.grimm', 'penrose.pm25', 'penrose.pm25.grimm', 'poal.pm10', 'poal.pm25', 'takapuna.pm10', 'takapuna.pm25', 'whangarei.pm10', 'whangaparoa.pm25', 'pakuranga.rain', 'penrose.rain', 'poal.rain', 'takapuna.rain', 'botanydowns.hum', 'gleneden.hum', 'henderson.hum', 'musickpt.hum', 'orewa.hum', 'pakuranga.hum', 'penrose.hum', 'poal.hum', 'takapuna.hum', 'penrose.so2', 'poal.so2', 'gleneden.solar', 'henderson.solar', 'musickpt.solar', 'orewa.solar', 'pakuranga.solar', 'penrose.solar', 'poal.solar', 'takapuna.solar', 'gleneden.temp', 'henderson.temp', 'musickpt.temp', 'orewa.temp', 'pakuranga.temp', 'penrose.temp', 'poal.temp', 'takapuna.temp', 'botanydowns.wind', 'gleneden.wind', 'henderson.wind', 'musickpt.wind', 'orewa.wind', 'pakuranga.wind', 'penrose.wind', 'poal.wind', 'takapuna.wind', 'botanydowns.windspeed', 'gleneden.windspeed', 'henderson.windspeed', 'pakuranga.windspeed', 'penrose.windspeed', 'poal.windspeed', 'takapuna.windspeed']

In [0]:
features = df[features_considered]
features.index = df['DateTime']
features.head()

How do these features vary across time?

In [0]:
features.plot(subplots=True)

As for the previous single location multi-variate time-series, the first step will be to normalize the dataset using the mean and standard deviation of the training data.

In [0]:
#dataset = features.values
#data_mean = dataset.mean(axis=0)
#data_std = dataset.std(axis=0)

In [0]:
#dataset = (dataset-data_mean)/data_std

### Single step model
In the single step setup, the model learns to predict a single point in the future based on some history provided.

The below function performs the same windowing task as before, however, here it samples the past observation based on the step size given.

In [0]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

The network is shown data from the last 720 observations that are sampled hourly. The sampling is done every one hour since a drastic change is not expected within 60 minutes. 

In [0]:
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

Let's check a single data point

In [0]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [0]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

In [0]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [0]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

Code after this point is untested/dev.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
#import tensorflow as tf
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import math
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2
from keras import backend as K

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import seaborn as sns
from string import ascii_letters
import IPython

import pickle

from keras.layers import Dense
import keras
from keras.models import Sequential
from keras.regularizers import l2

import scipy.io as sio

from sklearn import svm
from sklearn.metrics import accuracy_score


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://github.com/reganandrews/airdata/raw/master/master_data_fixed.csv.zip',
    fname='master_data_fixed.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

f = pd.read_csv(csv_path)

print('Loaded Data - Master File')

f.head()

In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://github.com/reganandrews/airdata/raw/master/location-gleneden.csv.zip',
    fname='location-gleneden.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

H1 = pd.read_csv(csv_path)

print('Loaded Data - Glen Eden')

H1.head()

In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://github.com/reganandrews/airdata/raw/master/location-henderson.csv.zip',
    fname='location-henderson.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

H2 = pd.read_csv(csv_path)

print('Loaded Data - Henderson')

H2.head()

In [0]:
# BEGIN Dataset

class DataSet(object):
    
    def __init__(self, images1, images2, labels, fake_data=False, one_hot=False,
                 dtype=tf.float32):
        """Construct a DataSet.
        one_hot arg is used only if fake_data is true.  `dtype` can be either
        `uint8` to leave the input as `[0, 255]`, or `float32` to rescale into
        `[0, 1]`.
        """
        dtype = tf.as_dtype(dtype).base_dtype
        if dtype not in (tf.uint8, tf.float32):
            raise TypeError('Invalid image dtype %r, expected uint8 or float32' % dtype)
        
        if fake_data:
            self._num_examples = 10000
            self.one_hot = one_hot
        else:
            assert images1.shape[0] == labels.shape[0], (
                'images1.shape: %s labels.shape: %s' % (images1.shape,
                                                        labels.shape))
            assert images2.shape[0] == labels.shape[0], (
                'images2.shape: %s labels.shape: %s' % (images2.shape,
                                                        labels.shape))
            self._num_examples = images1.shape[0]
            # Convert shape from [num examples, rows, columns, depth]
            # to [num examples, rows*columns] (assuming depth == 1)
            #assert images.shape[3] == 1
            #images = images.reshape(images.shape[0],
            #                        images.shape[1] * images.shape[2])
            if dtype == tf.float32 and images1.dtype != np.float32:
                # Convert from [0, 255] -> [0.0, 1.0].
                print("type conversion view 1")
                images1 = images1.astype(np.float32)
            
            if dtype == tf.float32 and images2.dtype != np.float32:
                print("type conversion view 2")
                images2 = images2.astype(np.float32)

        self._images1 = images1
        self._images2 = images2
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0
    
    @property
    def images1(self):
        return self._images1
    
    @property
    def images2(self):
        return self._images2
    
    @property
    def labels(self):
        return self._labels
    
    @property
    def num_examples(self):
        return self._num_examples
    
    @property
    def epochs_completed(self):
        return self._epochs_completed
    
    def next_batch(self, batch_size, fake_data=False):
        """Return the next `batch_size` examples from this data set."""
        if fake_data:
            fake_image = [1] * 784
            if self.one_hot:
                fake_label = [1] + [0] * 9
            else:
                fake_label = 0
            return [fake_image for _ in xrange(batch_size)], [fake_image for _ in xrange(batch_size)], [fake_label for _ in xrange(batch_size)]
        
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._images1 = self._images1[perm]
            self._images2 = self._images2[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        
        end = self._index_in_epoch
        return self._images1[start:end], self._images2[start:end], self._labels[start:end]

def read_mnist():

    data=sio.loadmat('MNIST.mat')
    
    train=DataSet(data['X1'],data['X2'],data['trainLabel'])
    
    tune=DataSet(data['XV1'],data['XV2'],data['tuneLabel'])
    
    test=DataSet(data['XTe1'],data['XTe2'],data['testLabel'])
    
    return train, tune, test


def read_xrmb():

    data=sio.loadmat('/share/data/speech-multiview/wwang5/cca/XRMBf2KALDI_window7_single.mat')
    
    train=DataSet(data['X1'],data['X2'],data['trainLabel'])
    
    tune=DataSet(data['XV1'],data['XV2'],data['tuneLabel'])
    
    test=DataSet(data['XTe1'],data['XTe2'],data['testLabel'])
    
    return train, tune, test

    
def read_flicker():

    data=sio.loadmat('/share/data/speech-multiview/wwang5/cca/VCCA/flicker/flicker_tensorflow_split1.mat')
    X1=data['X1']
    X2=data['X2']
    XV1=data['XV1']
    XV2=data['XV2']
    XTe1=data['XTe1']
    XTe2=data['XTe2']
    
    for i in range(2,11):
        
        data=sio.loadmat('/share/data/speech-multiview/wwang5/cca/VCCA/flicker/flicker_tensorflow_split' + str(i) + '.mat')
        
        X1=np.concatenate([X1, data['X1']])
        X2=np.concatenate([X2, data['X2']])
        XV1=np.concatenate([XV1, data['XV1']])
        XV2=np.concatenate([XV2, data['XV2']])
        XTe1=np.concatenate([XTe1, data['XTe1']])
        XTe2=np.concatenate([XTe2, data['XTe2']])
    
    train=DataSet(X1, X2, np.zeros(len(X1)))
    
    tune=DataSet(XV1, XV2, np.zeros(len(XV1)))
    
    test=DataSet(XTe1, XTe2, np.zeros(len(XTe1)))
    
    return train, tune, test

In [0]:
# BEGIN linear_cca.py
def linear_cca(H1, H2, outdim_size):
    """
    An implementation of linear CCA
    # Arguments:
        H1 and H2: the matrices containing the data for view 1 and view 2. Each row is a sample.
        outdim_size: specifies the number of new features
    # Returns
        A and B: the linear transformation matrices 
        mean1 and mean2: the means of data for both views
    """
    r1 = 1e-4
    r2 = 1e-4

    m = H1.shape[0]
    o = H1.shape[1]

    mean1 = numpy.mean(H1, axis=0)
    mean2 = numpy.mean(H2, axis=0)
    H1bar = H1 - numpy.tile(mean1, (m, 1))
    H2bar = H2 - numpy.tile(mean2, (m, 1))

    SigmaHat12 = (1.0 / (m - 1)) * numpy.dot(H1bar.T, H2bar)
    SigmaHat11 = (1.0 / (m - 1)) * numpy.dot(H1bar.T, H1bar) + r1 * numpy.identity(o)
    SigmaHat22 = (1.0 / (m - 1)) * numpy.dot(H2bar.T, H2bar) + r2 * numpy.identity(o)

    [D1, V1] = numpy.linalg.eigh(SigmaHat11)
    [D2, V2] = numpy.linalg.eigh(SigmaHat22)
    SigmaHat11RootInv = numpy.dot(numpy.dot(V1, numpy.diag(D1 ** -0.5)), V1.T)
    SigmaHat22RootInv = numpy.dot(numpy.dot(V2, numpy.diag(D2 ** -0.5)), V2.T)

    Tval = numpy.dot(numpy.dot(SigmaHat11RootInv, SigmaHat12), SigmaHat22RootInv)

    [U, D, V] = numpy.linalg.svd(Tval)
    V = V.T
    A = numpy.dot(SigmaHat11RootInv, U[:, 0:outdim_size])
    B = numpy.dot(SigmaHat22RootInv, V[:, 0:outdim_size])
    D = D[0:outdim_size]

    return A, B, mean1, mean2

In [0]:
# BEGIN model.py

# BEGIN model.py

def my_init_sigmoid(shape, dtype=None):
    rnd = K.random_uniform(
        shape, 0., 1., dtype)
    from keras.initializers import _compute_fans
    fan_in, fan_out = _compute_fans(shape)
    return 8. * (rnd - 0.5) * math.sqrt(6) / math.sqrt(fan_in + fan_out)

def my_init_others(shape, dtype=None):
    rnd = K.random_uniform(
        shape, 0., 1., dtype)
    from keras.initializers import _compute_fans
    fan_in, fan_out = _compute_fans(shape)
    return 2. * (rnd - 0.5) / math.sqrt(fan_in)

In [0]:
class DeepCCA():
    def __init__(self, layer_sizes1,
                 layer_sizes2, input_size1,
                 input_size2, outdim_size, reg_par, use_all_singular_values):

        self.layer_sizes1 = layer_sizes1  # [1024, 1024, 1024, outdim_size]
        self.layer_sizes2 = layer_sizes2
        self.input_size1 = input_size1
        self.input_size2 = input_size2
        self.outdim_size = outdim_size

        self.input_view1 = tf.placeholder(tf.float32, [None, input_size1])
        self.input_view2 = tf.placeholder(tf.float32, [None, input_size2])

        self.output_view1 = self.build_mlp_net(self.input_view1, layer_sizes1, reg_par)
        self.output_view2 = self.build_mlp_net(self.input_view2, layer_sizes2, reg_par)

        self.neg_corr = self.neg_correlation(self.output_view1, self.output_view2, use_all_singular_values)

    def build_mlp_net(self, input, layer_sizes, reg_par):
        output = input
        for l_id, ls in enumerate(layer_sizes):
            if l_id == len(layer_sizes) - 1:
                activation = None
                kernel_initializer = my_init_others
            else:
                activation = tf.nn.sigmoid
                kernel_initializer = my_init_sigmoid

            output = Dense(ls, activation=activation,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=l2(reg_par))(output)

        return output

    def neg_correlation(self, output1, output2, use_all_singular_values):
        r1 = 1e-4
        r2 = 1e-4
        eps = 1e-12

        # unpack (separate) the output of networks for view 1 and view 2
        H1 = tf.transpose(output1)
        H2 = tf.transpose(output2)

        m = tf.shape(H1)[1]

        H1bar = H1 - (1.0 / tf.cast(m, tf.float32)) * tf.matmul(H1, tf.ones([m, m]))
        H2bar = H2 - (1.0 / tf.cast(m, tf.float32)) * tf.matmul(H2, tf.ones([m, m]))

        SigmaHat12 = (1.0 / (tf.cast(m, tf.float32) - 1)) * tf.matmul(H1bar, tf.transpose(H2bar))
        SigmaHat11 = (1.0 / (tf.cast(m, tf.float32) - 1)) * tf.matmul(H1bar, tf.transpose(H1bar)) + r1 * tf.eye(self.outdim_size)
        SigmaHat22 = (1.0 / (tf.cast(m, tf.float32) - 1)) * tf.matmul(H2bar, tf.transpose(H2bar)) + r2 * tf.eye(self.outdim_size)

        # Calculating the root inverse of covariance matrices by using eigen decomposition
        [D1, V1] = tf.linalg.eigh(SigmaHat11)
        [D2, V2] = tf.linalg.eigh(SigmaHat22)

        # Added to increase stability
        posInd1 = tf.where(tf.greater(D1, eps))
        posInd1 = tf.reshape(posInd1, [-1, tf.shape(posInd1)[0]])[0]
        D1 = tf.gather(D1, posInd1)
        V1 = tf.gather(V1, posInd1)

        posInd2 = tf.where(tf.greater(D2, eps))
        posInd2 = tf.reshape(posInd2, [-1, tf.shape(posInd2)[0]])[0]
        D2 = tf.gather(D2, posInd2)
        V2 = tf.gather(V2, posInd2)

        SigmaHat11RootInv = tf.matmul(tf.matmul(V1, tf.linalg.diag(D1 ** -0.5)), tf.transpose(V1))
        SigmaHat22RootInv = tf.matmul(tf.matmul(V2, tf.linalg.diag(D2 ** -0.5)), tf.transpose(V2))

        Tval = tf.matmul(tf.matmul(SigmaHat11RootInv, SigmaHat12), SigmaHat22RootInv)

        if use_all_singular_values:
            # all singular values are used to calculate the correlation
            # corr = tf.sqrt(tf.linalg.trace(tf.matmul(tf.transpose(Tval), Tval)))  ### The usage of "sqrt" here is wrong!!!
            Tval.set_shape([self.outdim_size, self.outdim_size])
            s = tf.svd(Tval, compute_uv=False)
            corr = tf.reduce_sum(s)
        else:
            # just the top outdim_size singular values are used
            [U, V] = tf.linalg.eigh(tf.matmul(tf.transpose(Tval), Tval))
            non_critical_indexes = tf.where(tf.greater(U, eps))
            non_critical_indexes = tf.reshape(non_critical_indexes, [-1, tf.shape(non_critical_indexes)[0]])[0]
            U = tf.gather(U, non_critical_indexes)
            U = tf.gather(U, tf.nn.top_k(U[:, ]).indices)
            corr = tf.reduce_sum(tf.sqrt(U[0:self.outdim_size]))
        return -corr

In [0]:
# BEGIN train.py

n_epochs = 100
learning_rate = 0.01
momentum=0.99
batch_size = 800
outdim_size = 10
input_size1 = 784
input_size2 = 784
layer_sizes1 = [1024, 1024, 1024, outdim_size]
layer_sizes2 = [1024, 1024, 1024, outdim_size]
reg_par = 1e-4
use_all_singular_values = True


trainData, tuneData, testData = read_mnist()

dcca_model = DeepCCA(layer_sizes1, layer_sizes2,
                      input_size1, input_size2,
                      outdim_size,
                      reg_par, use_all_singular_values)


input_view1 = dcca_model.input_view1
input_view2 = dcca_model.input_view2
hidden_view1 = dcca_model.output_view1
hidden_view2 = dcca_model.output_view2
neg_corr = dcca_model.neg_corr


gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

train_op = tf.train.MomentumOptimizer(learning_rate, momentum).minimize(neg_corr,
                                                                        var_list=tf.trainable_variables())

tf.global_variables_initializer().run()


iterations = 0
for epoch in range(n_epochs):
    index = np.arange(trainData.num_examples)
    np.random.shuffle(index)
    trX1 = trainData.images1[index]
    trX2= trainData.images2[index]

    for start, end in zip(range(0, trainData.num_examples, batch_size),
            range(batch_size, trainData.num_examples, batch_size)):
        Xs1 = trX1[start:end]
        Xs2 = trX2[start:end]

        _, neg_corr_val = sess.run(
                                  [train_op, neg_corr],
                                   feed_dict={
                                       input_view1:Xs1,
                                       input_view2:Xs2
                                   })


        if iterations % 100 == 0:
            print("iteration:", iterations)
            print("neg_loss_for_train:", neg_corr_val)
            tune_neg_corr_val = sess.run(neg_corr,
                feed_dict={
                    input_view1: tuneData.images1,
                    input_view2: tuneData.images2
                })
            print("neg_loss_for_tune:", tune_neg_corr_val)

        iterations += 1


################# Linear CCA #############################

X1proj, X2proj = sess.run(
                        [hidden_view1, hidden_view2],
                        feed_dict={
                            input_view1: trainData.images1,
                            input_view2: trainData.images2
                        })
XV1proj, XV2proj = sess.run(
                        [hidden_view1, hidden_view2],
                        feed_dict={
                            input_view1: tuneData.images1,
                            input_view2: tuneData.images2
                        })
XTe1proj, XTe2proj = sess.run(
                        [hidden_view1, hidden_view2],
                        feed_dict={
                            input_view1: testData.images1,
                            input_view2: testData.images2
                        })
print("Linear CCA started!")
w = [None, None]
m = [None, None]
w[0], w[1], m[0], m[1] = linear_cca(X1proj, X2proj, 10)
print("Linear CCA ended!")
X1proj -= m[0].reshape([1, -1]).repeat(len(X1proj), axis=0)
X1proj = np.dot(X1proj, w[0])

XV1proj -= m[0].reshape([1, -1]).repeat(len(XV1proj), axis=0)
XV1proj = np.dot(XV1proj, w[0])

XTe1proj -= m[0].reshape([1, -1]).repeat(len(XTe1proj), axis=0)
XTe1proj = np.dot(XTe1proj, w[0])

trainLable = trainData.labels.astype('float')
tuneLable = tuneData.labels.astype('float')
testLable = testData.labels.astype('float')


################# SVM classify #############################

print('training SVM...')
clf = svm.LinearSVC(C=0.01, dual=False)
clf.fit(X1proj, trainLable.ravel())

p = clf.predict(XTe1proj)
test_acc = accuracy_score(testLable, p)
p = clf.predict(XV1proj)
valid_acc = accuracy_score(tuneLable, p)
print('DCCA: tune acc={}, test acc={}'.format(valid_acc, test_acc))